# This notebook is for data pre-processing

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import csv
import os
import re
from rapidfuzz import process, fuzz

## Data

In [ ]:
data = pd.read_csv('data.csv')
data.shape

## General Cleaning

In [ ]:
# Removing exact duplicate rows (rows identical across all columns)
# Count duplicates before removal
dup_count = data.duplicated().sum()
print(f"Found {dup_count} exact duplicate rows")
if dup_count > 0:
    # Drop exact duplicates and reset index
    data.drop_duplicates(inplace=True)
    data.reset_index(drop=True, inplace=True)
    print(f"Removed {dup_count} duplicates. New shape: {data.shape}")
else:
    print('No duplicate rows found.')

In [ ]:
# Detect groups where all attributes except RAM or storage (SSD/HDD) match,
# but RAM or storage is null in some rows and not in others.
# This helps find near-duplicates where only RAM/storage information is missing in some entries
cols = data.columns.tolist()
# flexible matching for RAM-like columns (case-insensitive, substring match)
ram_cols = [c for c in cols if 'ram' in c.lower() or 'memory' in c.lower()]
# storage detection expanded to SSD, HDD, storage, drive, disk
storage_cols = [c for c in cols if any(k in c.lower() for k in ['ssd', 'hdd', 'storage', 'drive', 'disk'])]
print('Detected ram-like columns:', ram_cols)
print('Detected storage-like columns (SSD/HDD):', storage_cols)
# Group by the specific keys you requested (flexible matching):
# price, date created, etat, model name, city, gpu, cpu, screen size, screen frequency, resolution
def find_col(variants, cols):
    """Return the first column whose name contains any of the provided variants (case-insensitive)."""
    variants = [v.lower() for v in variants]
    for c in cols:
        cname = c.lower()
        if any(v in cname for v in variants):
            return c
    return None
# mapping desired keys to lists of variants to match
desired_keys = {
    'price': ['price'],
    'date_created': ['date', 'create', 'created'],
    'etat': ['etat'],
    'model': ['model', 'model name', 'model_name'],
    'city': ['city'],
    'gpu': ['gpu', 'graphics'],
    'cpu': ['cpu', 'processor'],
    'screen_size': ['screen', 'size'],
    'screen_freq': ['frequency', 'hz', 'screen', 'refresh'],
    'resolution': ['resolution', 'res']
}
found_keys = {k: find_col(v, cols) for k, v in desired_keys.items()}
print('Found column mapping for grouping:')
for k, col in found_keys.items():
    print(f'  {k} -> {col}')
# Check for missing requested columns
missing = [k for k, col in found_keys.items() if col is None]
if missing:
    print('Warning: Could not find these requested grouping columns in the dataset:', missing)
# Build key_cols from the requested keys that were found (preserve order)
requested_order = ['price','date_created','etat','model','city','gpu','cpu','screen_size','screen_freq','resolution']
key_cols = [found_keys[k] for k in requested_order if found_keys.get(k)]
# If we don't have enough requested columns, fall back to grouping by all non-ram/non-storage cols
if len(key_cols) < 2:
    print('Not enough requested grouping columns found (need at least 2). Falling back to grouping by all non-ram/non-storage columns.')
    key_cols = [c for c in cols if c not in ram_cols + storage_cols]
print('Using key columns for grouping:', key_cols)
groups = data.groupby(key_cols, dropna=False)
problem_groups = []
for key, grp in groups:
    if len(grp) < 2:
        continue
    # check RAM columns for mixed null vs non-null within group
    ram_issue = False
    if ram_cols:
        ram_issue = any(grp[rc].isna().any() and (~grp[rc].isna()).any() for rc in ram_cols)
    # check storage columns (SSD/HDD) for mixed null vs non-null within group
    storage_issue = False
    if storage_cols:
        storage_issue = any(grp[sc].isna().any() and (~grp[sc].isna()).any() for sc in storage_cols)
    if ram_issue or storage_issue:
        problem_groups.append((key, grp))
# Auto-resolution: several possible strategies
# 1) If exactly one row has any storage info and it matches '1000' (HDD 1000), copy it to others and drop source
# 2) If multiple rows have storage info but across different storage columns (e.g., one row has SSD, another has HDD),
#    merge storage fields into a single target row and delete the redundant row(s).
resolved = 0
for key, grp in problem_groups:
    storage_existing = [c for c in storage_cols if c in grp.columns]
    if not storage_existing:
        continue
    # which rows in this group have any non-null storage value
    nonnull_mask = grp[storage_existing].notna().any(axis=1)
    rows_with_storage = grp[nonnull_mask]
    # Strategy A: single storage row -> maybe HDD 1000 pattern
    if len(rows_with_storage) == 1:
        src_idx = rows_with_storage.index[0]
        src_vals = rows_with_storage.iloc[0][storage_existing]
        def looks_like_1000(x):
            if pd.isna(x):
                return False
            s = str(x).lower().replace(' ', '')
            return ('1000' in s) or ('1tb' in s) or ('1000gb' in s)
        if any(looks_like_1000(v) for v in src_vals.dropna()):
            target_idxs = [i for i in grp.index if i != src_idx]
            for c in storage_existing:
                mask = data.loc[target_idxs, c].isna()
                if mask.any():
                    data.loc[target_idxs, c] = data.loc[target_idxs, c].where(~mask, other=data.loc[src_idx, c])
            data.drop(index=src_idx, inplace=True)
            resolved += 1
            print(f'Resolved group {key}: moved storage from index {src_idx} to {target_idxs} and deleted index {src_idx}')
        else:
            print(f'Group {key}: single storage row found but did not match 1000 pattern; skipped auto-resolution')
    # Strategy B: multiple rows with storage but complementary across columns -> merge into one row
    else:
        # determine non-null positions per storage column
        col_nonnull_counts = {c: grp[c].notna().sum() for c in storage_existing}
        total_nonnull_cells = sum(col_nonnull_counts.values())
        # If non-null cells are distributed across rows without overlap (each storage cell non-null appears in only one row),
        # we can merge them. Check that no row has two storage columns non-null (optional), and that total_nonnull_cells <= len(rows_with_storage) * len(storage_existing)
        # Simpler heuristic: if each storage column has at most one non-null entry and the number of rows_with_storage equals the number of distinct non-null rows,
        distinct_rows_with_storage = set(rows_with_storage.index.tolist())
        if all(v <= 1 for v in col_nonnull_counts.values()):
            # pick target as the first row that has any storage (prefer the one with SSD if present)
            target_idx = None
            # try to prefer a row that has SSD (or first storage column) non-null
            preferred = storage_existing[0] if storage_existing else None
            if preferred is not None:
                candidates = rows_with_storage[rows_with_storage[preferred].notna()].index.tolist()
                if candidates:
                    target_idx = candidates[0]
            if target_idx is None:
                target_idx = rows_with_storage.index[0]
            other_rows = [i for i in rows_with_storage.index if i != target_idx]
            # copy complementary storage values into target where null
            for other in other_rows:
                for c in storage_existing:
                    if pd.isna(data.at[target_idx, c]) and not pd.isna(data.at[other, c]):
                        data.at[target_idx, c] = data.at[other, c]
                # after copying, drop the other row
                data.drop(index=other, inplace=True)
                resolved += 1
                print(f'Merged storage from index {other} into {target_idx} and deleted {other} for group {key}')
        else:
            print(f'Group {key}: storage columns have multiple non-null entries per column; skipped auto-merge')
# After attempting auto-resolution, reset index if any changes were made
if resolved > 0:
    data.reset_index(drop=True, inplace=True)
    print(f'Auto-resolved {resolved} groups. New data shape: {data.shape}')
    # Recompute groups and problem_groups on the mutated dataframe so printed results reflect changes
    groups = data.groupby(key_cols, dropna=False)
    new_problem_groups = []
    for key, grp in groups:
        if len(grp) < 2:
            continue
        ram_issue = False
        if ram_cols:
            ram_issue = any(grp[rc].isna().any() and (~grp[rc].isna()).any() for rc in ram_cols)
        storage_issue = False
        if storage_cols:
            storage_issue = any(grp[sc].isna().any() and (~grp[sc].isna()).any() for sc in storage_cols)
        if ram_issue or storage_issue:
            new_problem_groups.append((key, grp))
    problem_groups = new_problem_groups
# Finally, print any remaining problem groups for manual review
if not problem_groups:
    print('No groups found where RAM or storage are null in some rows and not in others while other attributes match.')
else:
    print(f'Found {len(problem_groups)} potential groups (post-resolution):')
    for i, (key, grp) in enumerate(problem_groups, 1):
        print('\n---')
        print(f'Group {i} key: {key}')
        # show indices and relevant columns to help decide which rows to drop
        display_cols = key_cols + ram_cols + storage_cols
        # ensure ordering and existence
        display_cols = [c for c in display_cols if c in grp.columns]
        grp_display = grp[display_cols].copy()
        grp_display['_index'] = grp_display.index
        print(grp_display.to_string(index=False))
        print('---')

# Add a unique integer primary key column named 'id' (starting at 1). If 'id' exists, overwrite it after warning.
if 'id' in data.columns:
    print("Column 'id' already exists in the dataset; it will be overwritten with new sequential IDs.")
# Ensure index is contiguous before assigning IDs
data.reset_index(drop=True, inplace=True)
data.insert(0, 'id', range(1, len(data) + 1))
print(f"Added 'id' column as primary key. Data shape now: {data.shape}")

In [ ]:
data.shape

In [ ]:
# Export the cleaned data with IDs to CSV
data.to_csv('data_cleaned.csv', index=False)
print(f'Exported cleaned data to data_cleaned.csv with shape: {data.shape}')

## Merge cpu data into main data

In [ ]:
cpus_data = pd.read_csv('cpus.csv', on_bad_lines='warn')

# ------------------ NORMALIZATION ------------------

def normalize(s):
    if not s or pd.isna(s):
        return ''
    s = str(s).lower()
    s = re.sub(r'intel|processor|core|cpu', '', s)
    s = s.replace('-', ' ')
    s = re.sub(r'[^a-z0-9 ]+', ' ', s)
    return re.sub(r'\s+', ' ', s).strip()

# ------------------ TYPO/NEAR-MATCH CORRECTIONS ------------------

# Common typos and incomplete model numbers -> correct model
CPU_CORRECTIONS = {
    # 11th Gen typos (missing G7 suffix or wrong letter)
    'i5 1135u': 'i5-1135G7',
    'i5 1135': 'i5-1135G7',
    'i3 1115g7': 'i3-1115G4',  # G7 doesn't exist, it's G4
    'i3 1124g': 'i3-1125G4',   # typo
    'i3 1134g4': 'i3-1115G4',  # doesn't exist
    'i3 1145g4': 'i3-1115G4',  # doesn't exist
    
    # 12th Gen
    'i5 1244u': 'i5-1245U',    # typo
    'i5 1285p': 'i5-1240P',    # doesn't exist, closest is 1240P
    'i5 1235p': 'i5-1240P',    # P series
    'i5 12210u': 'i5-1235U',   # typo
    
    # 13th Gen
    'i7 13350u': 'i7-1355U',   # doesn't exist
    'i7 13340u': 'i7-1355U',   # doesn't exist
    'i7 1365p': 'i7-1360P',    # typo
    'i5 1345p': 'i5-1340P',    # typo
    
    # 8th Gen
    'i5 8300u': 'i5-8250U',    # 8300U doesn't exist
    'i5 8700': 'i5-8300H',     # desktop CPU, map to laptop equivalent
    'i5 8265': 'i5-8265U',     # missing U
    'i5 8600': 'i5-8300H',     # desktop
    'i5 8350 vpro': 'i5-8350U',
    'i5 8300 vpro': 'i5-8250U',
    'i5 8350de': 'i5-8350U',   # typo
    'i5 8635u': 'i5-8265U',    # doesn't exist
    'i7 8560u': 'i7-8550U',    # typo
    
    # 7th Gen
    'i5 7300': 'i5-7300U',     # missing suffix
    'i5 7300 vpro': 'i5-7300U',
    'i5 7400u': 'i5-7200U',    # 7400U doesn't exist
    'i5 7400': 'i5-7300HQ',    # desktop, map to laptop
    'i7 7375u': 'i7-7500U',    # doesn't exist
    
    # 6th Gen
    'i5 6300': 'i5-6300U',     # missing suffix
    'i7 6600': 'i7-6600U',     # missing suffix
    'i7 6600hq': 'i7-6700HQ',  # typo
    'i7 6850hq': 'i7-6820HQ',  # typo
    'i7 6550u': 'i7-6500U',    # typo
    'i3 6006': 'i3-6006U',     # missing suffix
    
    # 4th Gen
    'i7 4712': 'i7-4712MQ',    # missing suffix
    'i5 4570m': 'i5-4200M',    # 4570M doesn't exist
    'i3 4050u': 'i3-4030U',    # typo
    
    # 3rd Gen
    'i3 3220': 'i3-3120M',     # desktop, map to laptop
    'i3 3300': 'i3-3120M',     # doesn't exist
    
    # 2nd Gen
    'i5 2415m': 'i5-2410M',    # typo
    
    # 9th Gen
    'i7 9900': 'i7-9750H',     # desktop, map to laptop
    
    # Core M series
    'm3 7e': 'Core m3-7Y30',   # incomplete
    
    # Intel N-series
    'n200': 'Intel N200',
    'n4500': 'Intel Celeron N4500',
    
    # Misc
    '620': 'Intel Core 2 Duo T6600',  # very old, guess
}

def apply_cpu_corrections(normalized_cpu, original_cpu):
    """Apply known corrections for typos and incomplete model numbers."""
    # Check direct match
    if normalized_cpu in CPU_CORRECTIONS:
        return CPU_CORRECTIONS[normalized_cpu]
    
    # Check with gen prefix removed
    no_gen = re.sub(r'^\d+(?:th|nd|rd|st)?\s*gen\s*', '', normalized_cpu)
    if no_gen in CPU_CORRECTIONS:
        return CPU_CORRECTIONS[no_gen]
    
    return None


# ------------------ GENERIC CPU DETECTION ------------------

def detect_generic_cpu(cpu_name):
    """
    Detect generic CPU names like '11TH GEN INTEL CORE I5' or 'AMD RYZEN 5'
    Returns a tuple: (is_generic, brand, tier, generation) or (False, None, None, None)
    """
    if not cpu_name or pd.isna(cpu_name):
        return (False, None, None, None)
    
    s = str(cpu_name).lower().strip()
    
    # Skip Apple processors (not AMD A-series!)
    if 'apple' in s or 'bionic' in s or 'm1' in s or 'm2' in s or 'm3' in s:
        return (False, None, None, None)
    
    # Skip Intel Core Ultra (not AMD A-series!)
    if 'ultra' in s:
        # Intel Core Ultra - handle separately
        ultra_match = re.search(r'ultra\s*(\d)', s)
        if ultra_match:
            tier = f"ultra {ultra_match.group(1)}"
            # Check if it has a specific model number
            has_model = re.search(r'ultra\s*\d\s+\d{3}', s)
            if not has_model:
                return (True, 'intel', tier, None)
        return (False, None, None, None)
    
    # Intel N-series (N100, N200, N4500, N5100, etc.)
    if re.search(r'\bn[0-9]{3,4}\b', s):
        return (False, None, None, None)  # These are specific models, not generic
    
    # Intel patterns: "11th gen intel core i5", "intel core i7 12th gen", "core i5 11th"
    intel_pattern = re.search(
        r'(?:(\d{1,2})(?:th|nd|rd|st)?\s*gen(?:eration)?)?'  # optional gen prefix
        r'.*?(?:intel)?.*?(?:core)?\s*'
        r'(i[3579]|celeron|pentium|atom)'  # tier
        r'(?:\s*(\d{1,2})(?:th|nd|rd|st)?\s*gen(?:eration)?)?',  # optional gen suffix
        s
    )
    
    if intel_pattern:
        gen_prefix = intel_pattern.group(1)
        tier = intel_pattern.group(2)
        gen_suffix = intel_pattern.group(3)
        generation = gen_prefix or gen_suffix
        
        # Check if this is truly generic (no specific model number like 1135G7)
        # Generic: "11th gen i5", Non-generic: "i5-1135G7" or "i5 1135G7"
        has_model = re.search(r'i[3579]\s*[-]?\s*\d{4,5}', s)
        
        # For Pentium/Celeron, check for model numbers differently
        if tier in ['pentium', 'celeron']:
            has_model = re.search(r'(pentium|celeron)\s*(gold|silver)?\s*[a-z]?\d{4}', s)
        
        if tier and not has_model:
            return (True, 'intel', tier, generation)
    
    # AMD patterns: "AMD Ryzen 5", "Ryzen 7 5000 series", "AMD Ryzen 5 5000"
    # But NOT "AMD Ryzen 8845" which has a partial model number
    amd_pattern = re.search(
        r'(?:amd)?\s*ryzen\s*(\d)'  # Ryzen tier (3, 5, 7, 9)
        r'(?:\s*(\d{4})(?:\s*series)?)?',  # optional series like 5000, 7000
        s
    )
    
    if amd_pattern:
        tier = f"ryzen {amd_pattern.group(1)}"
        series = amd_pattern.group(2)
        
        # Check if this is truly generic (no specific model like 5600H or even partial 8845)
        # Match patterns like "ryzen 5 5600H" or "ryzen 8845"
        has_model = re.search(r'ryzen\s*\d\s+\d{4}[a-z]*', s)
        # Also check for pattern like "ryzen 8845" (no tier number)
        partial_model = re.search(r'ryzen\s+\d{4}', s)
        
        if has_model or partial_model:
            return (False, None, None, None)  # Not generic, has model number
        
        return (True, 'amd', tier, series)
    
    # AMD A-series: "AMD A10", "A6" - but NOT Apple or Intel Ultra
    # Must have explicit "amd" or "a" followed by number without other context
    amd_a_pattern = re.search(r'(?:^|\s)(?:amd\s+)?(a\d{1,2})(?:\s|$|-)', s)
    if amd_a_pattern and 'intel' not in s and 'apple' not in s and 'ultra' not in s:
        tier = amd_a_pattern.group(1)
        has_model = re.search(r'a\d{1,2}\s*[-]?\s*\d{4}', s)
        if not has_model:
            return (True, 'amd', tier, None)
    
    return (False, None, None, None)


def get_generic_cpu_stats(brand, tier, generation, cpus_df):
    """
    Get average stats for a generic CPU specification by filtering matching CPUs.
    """
    df = cpus_df.copy()
    df['name_lower'] = df['name'].str.lower()
    
    if brand == 'intel':
        if 'ultra' in tier:
            # Intel Core Ultra series
            ultra_num = tier.split()[-1]  # "ultra 7" -> "7"
            mask = df['name_lower'].str.contains(f'ultra\\s*{ultra_num}', regex=True, na=False)
        elif tier in ['pentium', 'celeron', 'atom']:
            # For Pentium/Celeron/Atom, just filter by name
            mask = df['name_lower'].str.contains(tier, regex=False, na=False)
            # Also filter to laptop category
            if 'cat' in df.columns:
                mask = mask & (df['cat'].str.lower() == 'laptop')
        else:
            # Core i3/i5/i7/i9
            tier_pattern = tier.replace('i', 'i[- ]?')  # handle i5, i-5, i 5
            mask = df['name_lower'].str.contains(tier_pattern, regex=True, na=False)
            
            # Filter by generation if specified
            if generation:
                gen = int(generation)
                if gen <= 10:
                    # Older gens: look for patterns like "i5-8250U" (8th gen starts with 8)
                    gen_pattern = rf'{tier_pattern}\s*[-]?\s*{gen}\d{{3}}'
                    mask = mask & df['name_lower'].str.contains(gen_pattern, regex=True, na=False)
                else:
                    # 11th gen+: look for patterns like "i5-1135G7" (11th gen starts with 11)
                    gen_pattern = rf'{tier_pattern}\s*[-]?\s*{gen}\d{{2}}'
                    mask = mask & df['name_lower'].str.contains(gen_pattern, regex=True, na=False)
    
    elif brand == 'amd':
        if 'ryzen' in tier:
            ryzen_num = tier.split()[-1]  # "ryzen 5" -> "5"
            mask = df['name_lower'].str.contains(f'ryzen\\s*{ryzen_num}', regex=True, na=False)
            
            # Filter by series if specified (5000, 7000, etc.)
            if generation:
                series = str(generation)
                # Match CPUs starting with that series number (5600, 5800, 7530, etc.)
                series_pattern = rf'ryzen\s*{ryzen_num}\s+{series[0]}\d{{3}}'
                mask = mask & df['name_lower'].str.contains(series_pattern, regex=True, na=False)
        else:
            # AMD A-series
            mask = df['name_lower'].str.contains(f'{tier}[-\\s]', regex=True, na=False)
    else:
        mask = pd.Series([False] * len(df))
    
    filtered = df[mask]
    
    if len(filtered) == 0:
        return None
    
    # Calculate median stats (more robust than mean for outliers)
    # Handle cpumark which might have commas
    def parse_cpumark(val):
        if pd.isna(val):
            return np.nan
        return float(str(val).replace(',', ''))
    
    cpumarks = filtered['cpumark'].apply(parse_cpumark)
    
    tdp_col = 'tdp(W)' if 'tdp(W)' in filtered.columns else 'tdp'
    
    # Get most common gpu_name for this tier
    if 'gpu_name' in filtered.columns:
        gpu_counts = filtered['gpu_name'].value_counts()
        common_gpu = gpu_counts.index[0] if len(gpu_counts) > 0 else 'NA'
    else:
        common_gpu = 'NA'
    
    return {
        'cpu_name': f"Generic {brand.upper()} {tier.upper()}" + (f" {generation}th Gen" if generation else ""),
        'cores': int(filtered['cores'].median()) if filtered['cores'].notna().any() else 'NA',
        'cpu_mark': int(cpumarks.median()) if cpumarks.notna().any() else 'NA',
        'tdp': round(filtered[tdp_col].median(), 1) if filtered[tdp_col].notna().any() else 'NA',
        'gpu_name': common_gpu,
        'match_count': len(filtered)
    }


# ------------------ PREPARE CLEANED CPUS ------------------

# Create normalized CPU names from cpus_data
cpus_data['norm'] = cpus_data['name'].apply(normalize)

# Get column names (handle different possible column names for tdp)
tdp_col = 'tdp(W)' if 'tdp(W)' in cpus_data.columns else 'tdp'

cpus = cpus_data[['name', 'norm', 'cores', 'cpumark', tdp_col]].copy()
cpus.columns = ['cpu_name', 'norm', 'cores', 'cpu_mark', 'tdp']
cpu_norms = cpus['norm'].tolist()

# Also create a dict for fast lookup by name
cpu_by_name = {row['cpu_name'].lower(): idx for idx, row in cpus.iterrows()}

# ------------------ NORMALIZE DATA CPU NAMES ------------------

# Find the CPU column in data
cpu_col = None
for col in ['cpu_name', 'CPU', 'cpu', 'Cpu']:
    if col in data.columns:
        cpu_col = col
        break

if cpu_col is None:
    raise ValueError("No CPU column found in data")

data['norm_cpu'] = data[cpu_col].apply(normalize)

# ------------------ MATCH & MAP ------------------

MATCH_THRESHOLD = 60

matched = unmatched = generic_matched = corrected = 0
scores = []
results = []

for idx, row in data.iterrows():
    original_cpu = row[cpu_col]
    n = row['norm_cpu']
    
    if not n:
        results.append({
            'mapped_cpu_name': 'NA',
            'match_score': 0,
            'cores': 'NA',
            'cpu_mark': 'NA',
            'tdp': 'NA',
            'gpu_name': 'NA',
            'match_type': 'empty'
        })
        unmatched += 1
        scores.append(0)
        continue

    # Step 1: Check for known typos/corrections
    correction = apply_cpu_corrections(n, original_cpu)
    if correction:
        # Try to find the corrected CPU in our database
        correction_norm = normalize(correction)
        match = process.extractOne(correction_norm, cpu_norms, scorer=fuzz.token_set_ratio)
        if match and match[1] >= 80:  # Higher threshold for corrections
            _, score, match_idx = match
            cpu = cpus.iloc[match_idx]
            results.append({
                'mapped_cpu_name': cpu['cpu_name'],
                'match_score': score,
                'cores': cpu['cores'],
                'cpu_mark': cpu['cpu_mark'],
                'tdp': cpu['tdp'],
                'gpu_name': cpus_data.iloc[match_idx].get('gpu_name', 'NA'),
                'match_type': f'corrected ({original_cpu} -> {correction})'
            })
            corrected += 1
            scores.append(score)
            continue

    # Step 2: Check if this is a generic CPU name
    is_generic, brand, tier, generation = detect_generic_cpu(original_cpu)
    
    if is_generic:
        generic_stats = get_generic_cpu_stats(brand, tier, generation, cpus_data)
        
        if generic_stats:
            results.append({
                'mapped_cpu_name': generic_stats['cpu_name'],
                'match_score': 100,  # Perfect match for generic
                'cores': generic_stats['cores'],
                'cpu_mark': generic_stats['cpu_mark'],
                'tdp': generic_stats['tdp'],
                'gpu_name': generic_stats['gpu_name'],
                'match_type': f'generic (based on {generic_stats["match_count"]} CPUs)'
            })
            generic_matched += 1
            scores.append(100)
            continue
        else:
            print(f'Generic CPU detected but no matching CPUs found: "{original_cpu}" ({brand} {tier} gen {generation})')

    # Step 3: Standard fuzzy matching
    match = process.extractOne(
        n,
        cpu_norms,
        scorer=fuzz.token_set_ratio
    )

    if match:
        _, score, match_idx = match
        scores.append(score)

        if score >= MATCH_THRESHOLD:
            cpu = cpus.iloc[match_idx]
            results.append({
                'mapped_cpu_name': cpu['cpu_name'],
                'match_score': score,
                'cores': cpu['cores'],
                'cpu_mark': cpu['cpu_mark'],
                'tdp': cpu['tdp'],
                'gpu_name': cpus_data.iloc[match_idx].get('gpu_name', 'NA'),
                'match_type': 'fuzzy'
            })
            matched += 1
        else:
            # Let's print the cpu name that didn't match well
            print(f'Unmatched CPU (score {score}): "{original_cpu}" normalized as "{n}"')
            results.append({
                'mapped_cpu_name': 'NA',
                'match_score': score,
                'cores': 'NA',
                'cpu_mark': 'NA',
                'tdp': 'NA',
                'gpu_name': 'NA',
                'match_type': 'unmatched'
            })
            unmatched += 1
    else:
        results.append({
            'mapped_cpu_name': 'NA',
            'match_score': 0,
            'cores': 'NA',
            'cpu_mark': 'NA',
            'tdp': 'NA',
            'gpu_name': 'NA',
            'match_type': 'no_match'
        })
        unmatched += 1
        scores.append(0)

# ------------------ MERGE RESULTS ------------------

results_df = pd.DataFrame(results)
data_merged = pd.concat([data.reset_index(drop=True), results_df], axis=1)

# Drop temporary column
data_merged = data_merged.drop(columns=['norm_cpu'])

# ------------------ SAVE OUTPUT ------------------

OUT_FN = 'data_with_cpus.csv'
data_merged.to_csv(OUT_FN, index=False)

# ------------------ REPORT ------------------

total = len(data)
avg_score = sum(scores) / len(scores) if scores else 0
print(
    f'\nWrote {OUT_FN} ({total} rows). '
    f'Fuzzy: {matched}, Generic: {generic_matched}, Corrected: {corrected}, Unmatched: {unmatched}, '
    f'Avg score: {avg_score:.1f}'
)

## Add gpu data to main data

In [ ]:
new_data = pd.read_csv(OUT_FN)

# Load cleaned GPUs reference
gpus_ref = pd.read_csv('gpus.csv')

# Normalization function for GPU names (same as in tools/map_gpus.py)
def normalize_gpu(s):
    if not s or pd.isna(s):
        return ''
    s = str(s).lower()
    s = re.sub(r'\b(nvidia|geforce|radeon|radeon pro|intel|graphics|gpu|mobile|laptop|pc|mx|with|max-q|ti|gtx|rtx|series|apple)\b', '', s)
    s = s.replace('-', ' ')
    s = re.sub(r'[^a-z0-9 ]+', ' ', s)
    return re.sub(r'\s+', ' ', s).strip()

# Build normalized GPU lookup
gpu_norms = gpus_ref['gpu_name'].apply(normalize_gpu).tolist()
gpu_data = gpus_ref.to_dict('records')

# Create a lookup dict by exact gpu_name for faster access
gpu_by_name = {g['gpu_name'].lower(): g for g in gpu_data}

# Best match function using rapidfuzz
def best_gpu_match(query, choices):
    if not query:
        return None
    match = process.extractOne(query, choices, scorer=fuzz.token_set_ratio)
    if match:
        return match  # (choice, score, idx)
    return None

# Apple GPU mapping based on CPU type
APPLE_GPU_MAP = {
    # M1 series - 8-core GPU (closest to 19-core performance tier)
    'm1': 'Apple 19-core GPU',
    'm1 pro': 'Apple 19-core GPU',
    'm1 max': 'Apple 38-core GPU',
    'm1 ultra': 'Apple 64-core GPU',
    # M2 series
    'm2': 'Apple 19-core GPU',
    'm2 pro': 'Apple 19-core GPU',
    'm2 max': 'Apple 38-core GPU',
    'm2 ultra': 'Apple 76-core GPU',
    # M3 series
    'm3': 'Apple 19-core GPU',
    'm3 pro': 'Apple 19-core GPU',
    'm3 max': 'Apple 38-core GPU',
    # M4 series
    'm4': 'Apple 19-core GPU',
    'm4 pro': 'Apple 38-core GPU',
    'm4 max': 'Apple 38-core GPU',
}

def get_apple_gpu_for_cpu(cpu_name):
    """Map Apple Silicon CPU to appropriate GPU benchmark entry."""
    if not cpu_name or pd.isna(cpu_name):
        return None
    cpu_lower = str(cpu_name).lower()
    
    # Check from most specific to least specific
    for pattern, gpu_name in sorted(APPLE_GPU_MAP.items(), key=lambda x: -len(x[0])):
        if pattern in cpu_lower:
            return gpu_name
    return None

# CPU-based GPU inference for CPUs with no gpu_name assigned
def infer_gpu_from_cpu(cpu_name):
    """Infer GPU from CPU name when no gpu_name was assigned."""
    if not cpu_name or pd.isna(cpu_name):
        return None
    cpu_lower = str(cpu_name).lower()
    
    # Qualcomm Snapdragon - use Adreno GPUs
    if 'snapdragon' in cpu_lower:
        if '8cx' in cpu_lower or '8c' in cpu_lower:
            return 'Adreno 680'  # High-end Snapdragon
        elif '7c' in cpu_lower:
            return 'Adreno 618'  # Mid-range
        else:
            return 'Adreno 618'  # Default Snapdragon
    
    # Intel Core 2 Duo / Core Duo - GMA integrated graphics
    if 'core 2 duo' in cpu_lower or 'core duo' in cpu_lower:
        return 'Intel GMA 4500MHD'  # Common integrated GPU for this era
    
    # Intel Celeron (old)
    if 'celeron' in cpu_lower and ('t3' in cpu_lower or 't1' in cpu_lower):
        return 'Intel GMA 4500MHD'
    
    # Generic Intel Core without model (like "INTEL CORE 620") - old laptop
    if 'intel' in cpu_lower and 'core' in cpu_lower:
        return 'Intel GMA 4500MHD'  # Assume old integrated graphics
    
    return None

# GPU mapping threshold
GPU_MATCH_THRESHOLD = 50

# Initialize new columns
new_data['gpu_match_score'] = np.nan
new_data['gpu_g3d_mark'] = np.nan
new_data['gpu_g2d_mark'] = np.nan
new_data['gpu_tdp'] = np.nan

# Map GPUs
dedicated_matched = 0
integrated_matched = 0
apple_matched = 0
inferred_matched = 0
gpu_unmatched = 0
gpu_scores = []

# Find the CPU column
cpu_col = None
for col in ['CPU', 'cpu', 'cpu_name', 'Cpu']:
    if col in new_data.columns:
        cpu_col = col
        break

for idx, row in new_data.iterrows():
    dedicated = row.get('DEDICATED_GPU')
    cpu_name = row.get(cpu_col) if cpu_col else None
    
    # Determine which GPU to look up
    if pd.isna(dedicated) or str(dedicated).strip() == '':
        # No dedicated GPU - look up the integrated GPU from gpu_name column
        gpu_to_match = row.get('gpu_name')
        is_dedicated = False
    else:
        # Has dedicated GPU - match the dedicated GPU
        gpu_to_match = dedicated
        is_dedicated = True
    
    # If no gpu_name, try to infer from CPU
    if (pd.isna(gpu_to_match) or str(gpu_to_match).strip() == '' or str(gpu_to_match).strip() == 'NA') and not is_dedicated:
        inferred_gpu = infer_gpu_from_cpu(cpu_name)
        if inferred_gpu:
            gpu_to_match = inferred_gpu
            new_data.at[idx, 'gpu_name'] = inferred_gpu
    
    # Skip if still no GPU to match
    if pd.isna(gpu_to_match) or str(gpu_to_match).strip() == '' or str(gpu_to_match).strip() == 'NA':
        gpu_unmatched += 1
        continue
    
    # Special handling for generic "Apple GPU"
    if 'apple gpu' in str(gpu_to_match).lower():
        apple_gpu = get_apple_gpu_for_cpu(cpu_name)
        if apple_gpu and apple_gpu.lower() in gpu_by_name:
            g = gpu_by_name[apple_gpu.lower()]
            new_data.at[idx, 'gpu_name'] = g['gpu_name']
            new_data.at[idx, 'gpu_match_score'] = 100
            new_data.at[idx, 'gpu_g3d_mark'] = g.get('g3d_mark', None)
            new_data.at[idx, 'gpu_g2d_mark'] = g.get('g2d_mark', None)
            new_data.at[idx, 'gpu_tdp'] = g.get('tdp(w)', None)
            apple_matched += 1
            gpu_scores.append(100)
            continue
    
    # Normalize and match GPU
    norm_gpu = normalize_gpu(gpu_to_match)
    if not norm_gpu:
        gpu_unmatched += 1
        continue
    
    match = best_gpu_match(norm_gpu, gpu_norms)
    if match:
        choice, score, match_idx = match
        gpu_scores.append(score)
        if score >= GPU_MATCH_THRESHOLD:
            g = gpu_data[match_idx]
            # Update gpu_name only if it's a dedicated GPU or was inferred
            if is_dedicated:
                new_data.at[idx, 'gpu_name'] = g['gpu_name']
                dedicated_matched += 1
            else:
                integrated_matched += 1
            # Always fill the benchmark attributes
            new_data.at[idx, 'gpu_match_score'] = score
            new_data.at[idx, 'gpu_g3d_mark'] = g.get('g3d_mark', None)
            new_data.at[idx, 'gpu_g2d_mark'] = g.get('g2d_mark', None)
            new_data.at[idx, 'gpu_tdp'] = g.get('tdp(w)', None)
        else:
            # Low score - keep the inferred/assigned name but mark as unmatched
            gpu_unmatched += 1
    else:
        gpu_unmatched += 1
        gpu_scores.append(0)

# Report
avg_gpu_score = sum(gpu_scores) / len(gpu_scores) if gpu_scores else 0
print(f'GPU Mapping: Dedicated: {dedicated_matched}, Integrated: {integrated_matched}, Apple: {apple_matched}, Unmatched: {gpu_unmatched}')
print(f'Avg score: {avg_gpu_score:.1f}')
print(f'\nSample gpu_name values after mapping:')
print(new_data['gpu_name'].dropna().value_counts().head(15))
print(f'\nGPU benchmark columns filled: {new_data["gpu_g3d_mark"].notna().sum()} rows')

# Show remaining unmatched
still_unmatched = new_data[new_data['gpu_g3d_mark'].isna()]
if len(still_unmatched) > 0:
    print(f'\nRemaining unmatched ({len(still_unmatched)} rows):')
    print(still_unmatched[['gpu_name', cpu_col]].head(20) if cpu_col else still_unmatched[['gpu_name']].head(20))

# Export
new_data.to_csv('data_with_cpus_gpus.csv', index=False)